# **Consensus construction from https://github.com/friburgo-moc/FidelityFinder**

In [ ]:
#@markdown # Upload your FASTA file
#@markdown - The input must be a fasta file with the sequences where you want to find barcodes and build consensus sequences\
#@markdown - The purpose of this code is to find barcodes and build consensus from small fasta files or with the intention of testing the program. For large fasta files we recommend to follow the nextflow pipeline available at https://github.com/friburgo-moc/FidelityFinder.
from google.colab import files
uploaded = files.upload()
fasta_file = list(uploaded.keys())[0]

Saving Test_bcfinder2_qual_filtered.fasta to Test_bcfinder2_qual_filtered.fasta


In [ ]:
#@markdown # Input parameters
#@markdown
#@markdown - Length of the barcode used
bc_size = 12 #@param {type:"integer"}
#@markdown - Inferior cut-off value (barcodes with this frequency or lower will not be used to build consensus sequences)
inferior = 2 # @param {type:"integer"}
#@markdown - Superior cut-off value (barcodes with this frequency or higher will not be used to build consensus sequences)
superior = 1000000 # @param {type:"integer"}
#@markdown - Threshold used to build consensus (0 - 1). For example, with a threshold of 0.5, a nucleotide must be present in at least 50% of the sequences that share a barcode to appear in the consensus sequence; otherwise, an N (indeterminate base) will be used
threshold_consensus = 0.9 # @param {type:"number"}
#@markdown - Prefix used in the name of output files generated
out_prefix = "undefined" #@param {type:"string"}
#@markdown \
#@markdown
#@markdown ## Two options to find a barcode:
#@markdown  - Select this option if you want to find barcodes in a fast way. You have to provide the sequence (~20-30 nt to ensure it is unique) prior to the barcode (5'-->3')
fast_barcode_finding = False #@param {type:"boolean"}
primer = 'GGAATGGATGGCCCAAAAGTTAAACTG' #@param {type:"string"}
#@markdown  - Select this option if you want to find barcodes in a slower way, but with higher accuraccy. You have to provide the sequence of the insert of your library indicating each nucleotide of the barcode as "N"
slow_barcode_finding = True #@param {type:"boolean"}
insert_sequence = 'CAGGAGCCGATAGACAAGGAACTGTATCCTTTAACTTCCCTCAGATGGATCCACTCTTTGGCAACGACCCCTCGTCACAATAAAGATAGGGGGGCAACTAAAGGAAGCTCTATTAGATACAGGAGCAGATGATACAGTATTAGAAGAAATGAGTTTGCCAGGAAGATGGAAACCAAAAATGATAGGGGGAATTGGAGGTTTTATCAAAGTAAGACAGTATGATCAGATACTCATAGAAATCTGTGGACATAAAGCTATAGGTACAGTATTAGTAGGACCTACACCTGTCAACATAATTGGAAGAAATCTGTTGACTCAGATTGGTTGCACTTTAAATTTTCCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAATGGATGGCCCAAAAGTTAAACTGNNNNNNNNNNNNGAAGAG' #@param {type:"string"}


#examples of inser sequences:
#CTTCCTACAAGGGAAGGCCAGGGAATTTTCTTCAGAGCAGACCAGAGCCAACAGCCCCACCAGAAGAGAGCTTCAGGTCTGGGGTAGAGACAACAACTCCCCCTCAGAAGCAGGAGCCGATAGACAAGGAACTGTATCCTTTAACTTCCCTCAGATGGATCCACTCTTTGGCAACGACCCCTCGTCACAATAAAGATAGGGGGGCAACTAAAGGAAGCTCTATTAGATACAGGAGCAGATGATACAGTATTAGAAGAAATGAGTTTGCCAGGAAGATGGAAACCAAAAATGATAGGGGGAATTGGAGGTTTTATCAAAGTAAGACAGTATGATCAGATACTCATAGAAATCTGTGGACATAAAGCTATAGGTACAGTATTAGTAGGACCTACACCTGTCAACATAATTGGAAGAAATCTGTTGACTCAGATTGGTTGCACTTTAAATTTTCCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAATGGATGGCCCAAAAGTTAAACNNNNNNNNNNNNNNACCTT
#CAGGAGCCGATAGACAAGGAACTGTATCCTTTAACTTCCCTCAGATGGATCCACTCTTTGGCAACGACCCCTCGTCACAATAAAGATAGGGGGGCAACTAAAGGAAGCTCTATTAGATACAGGAGCAGATGATACAGTATTAGAAGAAATGAGTTTGCCAGGAAGATGGAAACCAAAAATGATAGGGGGAATTGGAGGTTTTATCAAAGTAAGACAGTATGATCAGATACTCATAGAAATCTGTGGACATAAAGCTATAGGTACAGTATTAGTAGGACCTACACCTGTCAACATAATTGGAAGAAATCTGTTGACTCAGATTGGTTGCACTTTAAATTTTCCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAATGGATGGCCCAAAAGTTAAACTGNNNNNNNNNNNNGAAGAG

In [ ]:
#@markdown # Main program

# Imports
!pip install biopython
!apt-get install -qq -y mafft
import sys, os, math
import json
import seaborn as sbn
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
from Bio import SeqIO
from collections import defaultdict
from scipy import stats
from Bio.Align.Applications import MafftCommandline
from io import StringIO
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Align.AlignInfo import SummaryInfo
from tqdm import tqdm

########################################################################################
print()
print("\t", "*" * 75)
print()
print("\tPython script to calculate consensus sequence by barcode")
print()
print("\t-. .-.   .-. .-.   .-. .-.   .-. .-.   .-. .-.   .-. .-.   .-. .-.   .")
print("\t  \   \ /   \   \ /   \   \ /   \   \ /   \   \ /   \   \ /   \   \ / ")
print("\t / \   \   / \   \   / \   \   / \   \   / \   \   / \   \   / \   \  ")
print("\t~   `-~ `-`   `-~ `-`   `-~ `-`   `-~ `-`   `-~ `-`   `-~ `-`   `-~ `-")
print()
print("\t", "*" * 75, "\n")
########################################################################################

__doc__ = """
SYNOPSIS

 Python script to calculate consensus sequences by barcode


AUTHORS

    This is a script modified by Javier Martinez del Río (javier.martinez@cbm.csic.es) and
    Estrella Frutos-Beltrán (efrutos@cbm.csic.es) from the version 1 of the script created
    by the following authors:

    Genomics & NGS Facility (CBMSO-CSIC) | Eva Castillo    <ecastillo@cbm.csic.es>
                                         | Eva Sacristán  <esacristan@cbm.csic.es>
                                         | Sandra González  <sandra.g@cbm.csic.es>
                                         | Ramón Peiró-Pastor <rpeiro@cbm.csic.es>

LICENSE

    Copyright (c) 2018 CBMSO

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""
__author__ = "Eva Castillo, Eva Sacristán, Sandra González, Ramón Peiró-Pastor, Javier Martínez del Río, Estrella Frutos-Beltrán"
__version__ = 'v2.0.0'

## Functions
##----------


def send_error_message(message):
    """Gives an error message if argument is not valid"""
    print("\n\t########## ERROR ########## \n{}\n".format(message))
    sys.exit(1)


def find_barcode(reference_sequence, query_sequence):
    # reference_sequence is the insert of the library sequenced indicating each letter of the barcode zone as "N"
    # query_sequence is the raw sequence of the insert obtained by NGS
    # Align reference and query sequences:
    mafft_cline = MafftCommandline("mafft", input="-", auto=True, thread=30)
    stdout, stderr = mafft_cline(stdin=f">reference_sequence\n{reference_sequence}\n>query_sequence\n{query_sequence}")
    align = AlignIO.read(StringIO(stdout), "fasta")
    ref_aligned = str(align[0].seq)
    query_aligned = str(align[1].seq)
    # Select barcode sequence (it corresponds to the part of query_sequence that aligns with the letters "N" of reference_sequence)
    barcode = ""
    for ref_nuc, query_nuc in zip(ref_aligned, query_aligned):
        if ref_nuc == "n" and  query_nuc != "-":
            barcode += query_nuc
    return barcode



## Main program
##-------------
print("\tStep 1 => Open & read FASTA file | Barcode search")

if fast_barcode_finding == True:
  print("Using fast way to find barcodes")
  fasta_hand = SeqIO.parse(fasta_file, "fasta")
  fasta_dic = defaultdict(list)
  bc_lengths_dic = defaultdict(int)
  accepted_sequences = 0
  rejected_sequences = 0
  for record in tqdm(fasta_hand):
    read     = str(record.seq)
    primer_index   = read.find(primer)
    if primer_index >= 0:
      accepted_sequences += 1
      bc_start = primer_index + len(primer)
      bc_end   = bc_start + bc_size
      barcode  = read[bc_start:bc_end]
      bc_lengths_dic[len(barcode)] += 1
      fasta_dic[barcode].append(read)
    else:
      rejected_sequences += 1
  print("\t\tTotal initial sequences", accepted_sequences + rejected_sequences)
  print("\t\tAccepted sequences", accepted_sequences)
  print("\t\tRejected sequences (primer not found)", rejected_sequences)
  print("\t\tBarcodes lengths and their frequency: ", dict(bc_lengths_dic))

elif slow_barcode_finding == True:
  print("Using slow way to find barcodes")
  fasta_hand = SeqIO.parse(fasta_file, "fasta")
  fasta_dic = defaultdict(list)
  bc_lengths_dic = defaultdict(int)
  accepted_sequences = 0
  rejected_sequences_bc_size = 0
  rejected_sequences_no_align = 0
  for record in tqdm(fasta_hand):
    read     = str(record.seq)
    try:
      barcode  = find_barcode(insert_sequence, read)
      bc_lengths_dic[len(barcode)] += 1
      if len(barcode) == bc_size:
          fasta_dic[barcode].append(read)
          accepted_sequences += 1
      else:
          rejected_sequences_bc_size += 1
    except Exception as e:
      rejected_sequences_no_align += 1
      print(e)
  print("\t\tTotal initial sequences: ", accepted_sequences + rejected_sequences_bc_size + rejected_sequences_no_align)
  print("\t\tAccepted sequences: ", accepted_sequences)
  print("\t\tRejected sequences (different barcode length): ", rejected_sequences_bc_size)
  print("\t\tRejected sequences (no alignment to the reference sequence): ", rejected_sequences_no_align)
  print("\t\tBarcodes lengths and their frequency: ", dict(bc_lengths_dic))
  print("\t\tTotal unique barcodes with expected length found: ", len(fasta_dic.keys()))

else:
  send_error_message("Please, select an option fo find barcodes")


if accepted_sequences < 1:
    send_error_message(f'No sequences could be found (sample: {out_prefix}), please check the input parameters\nHave you chosen the right input parameters?')

print("\tStep 1 done\n")





print("\tStep 2 => Calculating consensus sequences")

# files
fasta_tmp = "tmp.fa"
xls_file                = out_prefix + "_consensus.xls"
discarded_file          = out_prefix + "_discarded.txt"
profiles_file           = out_prefix + "_consensus.prf"
consensus_file          = out_prefix + "_consensus.fna"
barcodes_file           = out_prefix + "_barcodes.json"

# handlers
xls_hand                = open(xls_file, "w")
discarded_hand          = open(discarded_file, "w")
prf_hand                = open(profiles_file, "w")
consensus_hand          = open(consensus_file, "w")
barcodes_hand           = open(barcodes_file, 'w')

# header for xls file
header_string = "#barcode\tconsensus\tnum_seqs\n"
xls_hand.write(header_string)

# store info to draw plot
plot_dic  = {}
plot2_dic = {}
barcodes_dic = {}

for barcode in tqdm(fasta_dic.keys()):
  counter  = 0
  seqList  = fasta_dic[barcode]
  nseqs    = len(seqList)
  barcodes_dic[barcode] = nseqs
  try:
    plot_dic[nseqs] += 1
  except KeyError:
    plot_dic[nseqs] = 1
  if nseqs>inferior and nseqs<superior:
    try:
      plot2_dic[nseqs] += 1
    except KeyError:
      plot2_dic[nseqs] = 1
    tmp_hand = open(fasta_tmp, "w")
    for seq in seqList:
      string = ">" + barcode + "_" + str(counter) + "\n" + seq + "\n"
      tmp_hand.write(string)
      counter += 1
    tmp_hand.close()
    mafft_cline = MafftCommandline("mafft", input=fasta_tmp, thread=30)
    stdout, stderror= mafft_cline()
    align = AlignIO.read(StringIO(stdout), "fasta")
    align_sum = SummaryInfo(align)
    consensus = str(SummaryInfo.gap_consensus(align_sum, threshold=threshold_consensus, ambiguous='N')).upper()
    matrix = str(SummaryInfo.pos_specific_score_matrix(align_sum)).upper()
    os.remove(fasta_tmp)
    string = ">" + barcode + " nseqs=" + str(nseqs) + "\n" + consensus + "\n"
    consensus_hand.write(string)
    string = "barcode=" + barcode + " nseqs=" + str(nseqs) + "\n" + matrix + "\n"
    prf_hand.write(string)
    string = barcode + "\t" + consensus + "\t" + str(nseqs) + "\n"
    xls_hand.write(string)
  else:
    string = "barcode=" + barcode + " nseqs=" + str(nseqs) + "\n"
    discarded_hand.write(string)

json.dump(barcodes_dic, barcodes_hand)
barcodes_hand.close()
consensus_hand.close()
prf_hand.close()
xls_hand.close()
discarded_hand.close()

print("\t\tSequences per barcode and their respective frequencies: ", plot_dic)
print(f"\t\tTotal number of consensus sequences constructed (using {inferior} as inferior cutoff value): ", sum(plot2_dic.values()))
if len(plot2_dic.keys()) < 1:
    send_error_message(f'No consensus sequences could be constructed using {inferior} as inferior cutoff value')

print("\tStep 2 done\n")



print("\tStep 3 => Drawing plot")
plot_file  =  out_prefix + "_consensus.png"
x = plot_dic.keys()
y = plot_dic.values()
plt.scatter(x, y, marker="o")
plt.yscale('log')
titlename = 'Primer ID distribution for ' + out_prefix + ' reads (no cut-off)'
plt.title(titlename)
plt.xlabel('Raw Sequence Reads per Unique Primer ID')
plt.ylabel('Number of Distinct Primer IDs')
plt.savefig(plot_file)
plt.close()

plot2_file  = out_prefix + "_cutoff_consensus.png"
x = plot2_dic.keys()
y = plot2_dic.values()
plt.scatter(x, y, marker="o")
plt.yscale('log')
titlename = 'Primer ID distribution for ' + out_prefix + ' reads cut-off (' + str(inferior) + ", " + str(superior) + ")"
plt.title(titlename)
plt.xlabel('Raw Sequence Reads per Unique Primer ID')
plt.ylabel('Number of Distinct Primer IDs')
plt.savefig(plot2_file)
plt.close()

print("\tStep 3 done\n")

print("\tJOB DONE !!! Have a nice & pythonic day!!! (^_^)\n")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.2 MB/s eta 0:00:00
Extracting templates from packages: 100%
Selecting previously unselected package fonts-lato.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../00-fonts-lato_2.0-2.1_all.deb ...
Unpacking fonts-lato (2.0-2.1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../01-netbase_6.3_all.deb ...
Unpacking netbase (6.3) ...
Selecting previously unselected package libclone-perl.
Preparing to unpack .../02-libclone-perl_0.45-1build3_amd64.deb ...
Unpacking libclone-perl (0.45-1build3) ...
Selecting previously unselected package libdata-dump-perl.
Preparing to unpack .../03-libdata-dump-perl_1.25-1_all.deb ...
Unpacking libdata-dump-perl (1.25-1) ...
Selecting previously unselected package libencode-locale-perl.
Preparing to unpack .../04-libencode-locale-perl_1.05-1.1_all.deb ...
Unpacking libencode-locale-perl (1.05-1.1) ...
Selecting previously uns

227it [00:20, 11.24it/s]


		Total initial sequences:  227
		Accepted sequences:  227
		Rejected sequences (different barcode length):  0
		Rejected sequences (no alignment to the reference sequence):  0
		Barcodes lengths and their frequency:  {12: 227}
		Total unique barcodes with expected length found:  56
	Step 1 done

	Step 2 => Calculating consensus sequences


100%|██████████| 56/56 [00:02<00:00, 25.75it/s]


		Sequences per barcode and their respective frequencies:  {1: 34, 2: 9, 10: 2, 26: 1, 40: 1, 8: 1, 3: 4, 14: 1, 42: 1, 4: 1, 9: 1}
		Total number of consensus sequences constructed (using 2 as inferior cutoff value):  13
	Step 2 done

	Step 3 => Drawing plot
	Step 3 done

	JOB DONE !!! Have a nice & pythonic day!!! (^_^)



In [ ]:
#@markdown # Download files
files.download(xls_file)
files.download(discarded_file)
files.download(profiles_file)
files.download(consensus_file)
files.download(barcodes_file)
files.download(plot_file)
files.download(plot2_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>